In [1]:
# Dependencies
from bs4 import BeautifulSoup as bs
import pandas as pd
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager
import re
import csv


In [2]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 90.0.4430
[WDM] - Get LATEST driver version for 90.0.4430


[WDM] - Get LATEST driver version for 90.0.4430
[WDM] - Trying to download new driver from https://chromedriver.storage.googleapis.com/90.0.4430.24/chromedriver_win32.zip
[WDM] - Driver has been saved in cache [C:\Users\galci\.wdm\drivers\chromedriver\win32\90.0.4430.24]


In [3]:
# URL of page to be scraped nasa mars news
url = 'https://finances.worldbank.org/Other/IEG-World-Bank-Project-Performance-Ratings/rq9d-pctf'
browser.visit(url)
html = browser.html
soup = bs(html, 'html.parser')
#print(soup.prettify())

In [4]:
# get news title, news link, news text, news date
results = soup.find_all("div", attrs={"class":"socrata-table"})

In [5]:
table = results[0].find("table")

In [6]:
body = table.find_all("tr")
head=body[0]
body_rows=body[1:]
headings=[]
for item in head.find_all("th"):
    item=(item.text).rstrip("\n")
    headings.append(item)
#print(headings)

In [7]:
all_rows = [] # will be a list for list for all rows
for row_num in range(len(body_rows)): # A row at a time
    row = [] # this will old entries for one row
    for row_item in body_rows[row_num].find_all("td"): #loop through all row entries
        # row_item.text removes the tags from the entries
        # the following regex is to remove \xa0 and \n and comma from row_item.text
        # xa0 encodes the flag, \n is the newline and comma separates thousands in numbers
        aa = re.sub("(\xa0)|(\n)|,","",row_item.text)
        #append aa to row - note one row entry is being appended
        row.append(aa)
    # append one row to all_rows
    all_rows.append(row)

In [8]:
project_df = pd.DataFrame(data=all_rows,columns=headings)

In [9]:
#print(df.columns)

In [10]:
project_df.columns=['Project ID','Project Name','Region','Country Code','Country Name','Approval Date',
       'Approval FY','Sector Board','Agreement Type','Lending Project Cost','Net Commitment',
       'Deactivation Date','Exit FY','Lending Instrument Typen','Lending Instrument','Product Line Code',
       'Product Line','IEG_EvalDate','IEG_EvalFY','IEG_EvalType','ERR at Appraisal','ERR at Completion',
       'IEG_Outcome','IEG_RDO','(disc)IEG_IDImpact','IEG_BankQualityAtEntry','IEG_BankQualityOfSupervision',
       'IEG_OverallBankPerf','(disc)IEG_BorrPrep','EG_ImplementingAgencyPerf','IEG_GovernmentPerf',
       'IEG_OverallBorrPerf','IEG_ICRQuality','(disc)IEG_Sustainability','IEG_MEQualit','IEG_SourceDocumentURL']

In [11]:
project_df=project_df.loc[:,['Project ID','Project Name','Region','Country Code','Country Name','Approval Date',
       'Approval FY','Sector Board','Lending Project Cost','Net Commitment',
       'Deactivation Date','Lending Instrument','Product Line Code',
       'Product Line','IEG_Outcome','IEG_SourceDocumentURL']]
project_df

,Project ID,Project Name,Region,Country Code,Country Name,Approval Date,Approval FY,Sector Board,Lending Project Cost,Net Commitment,Deactivation Date,Lending Instrument,Product Line Code,Product Line,IEG_Outcome,IEG_SourceDocumentURL
0,P000001,3A-CB Nat Res & Wildlife Mgmt SIL (F...,AFR,3A,Africa,9/14/1995,1996,Environment,7000000,-31988.17,6/30/2004,SIL,GE,Global Environment Project,Unsatisfactory,http://lnweb90.worldbank.org/oed/oeddoclib.nsf...
1,P000003,REIMP(CEN.ENV.INFO),AFR,3A,Africa,12/18/1997,1998,Environment,19700000,0,6/30/2003,SIL,GE,Global Environment Project,Satisfactory,http://lnweb90.worldbank.org/oed/oeddoclib.nsf...
2,P000010,BOAD,AFR,3A,Africa,2/1/1990,1990,Financial Sector,,44669376.09,12/31/1997,FIL,PE,IBRD/IDA,Moderately Satisfactory,http://lnweb90.worldbank.org/oed/oeddoclib.nsf...
3,P000017,3A-TG/BN Engineering TAL (FY92),AFR,3A,Africa,5/19/1992,1992,Energy and Mining,6100000,5500000,12/30/1999,TAL,PE,IBRD/IDA,Unsatisfactory,http://lnweb90.worldbank.org/oed/oeddoclib.nsf...
4,P000034,INFRASTRUCTURE REHAB,AFE,AO,Angola,7/16/1991,1992,Transport,32100000,30704714,3/31/1999,TAL,PE,IBRD/IDA,Moderately Satisfactory,http://lnweb90.worldbank.org/oed/oeddoclib.nsf...
5,P000035,LOBITO/BENG.REHAB.,AFE,AO,Angola,1/7/1992,1992,Urban Development,65000000,45600000,9/28/2001,SIM,PE,IBRD/IDA,Satisfactory,http://lnweb90.worldbank.org/oed/oeddoclib.nsf...
6,P000037,ECON FIN MGMT I,AFE,AO,Angola,6/19/1991,1991,Economic Policy,26100000,23000000,12/31/1998,TAL,PE,IBRD/IDA,Unsatisfactory,http://lnweb90.worldbank.org/oed/oeddoclib.nsf...
7,P000040,POWER SECTOR REHAB.,AFE,AO,Angola,6/11/1992,1992,Energy and Mining,33500000,18052817.8,12/30/1999,SIM,PE,IBRD/IDA,Unsatisfactory,http://lnweb90.worldbank.org/oed/oeddoclib.nsf...
8,P000042,EDUCATION I,AFE,AO,Angola,6/2/1992,1992,Education,27100000,27100000,9/30/1999,SIL,PE,IBRD/IDA,Satisfactory,http://lnweb90.worldbank.org/oed/oeddoclib.nsf...
9,P000044,FINANCIAL INSTITUTIO,AFE,AO,Angola,9/3/1992,1993,Financial and Private Sector Develop...,21000000,21000000,4/30/2002,TAL,PE,IBRD/IDA,Satisfactory,http://lnweb90.worldbank.org/oed/oeddoclib.nsf...


In [12]:
project_df.to_html('app/templates/iegData.html')

In [13]:
browser.quit()

In [14]:
gdp_df=pd.read_csv('data/gdp-per-capita-worldbank.csv')
gdp_df.head(20)

,Entity,Code,Year,"GDP per capita, PPP (constant 2011 international $)"
0,Afghanistan,AFG,2002,1063.635574
1,Afghanistan,AFG,2003,1099.194507
2,Afghanistan,AFG,2004,1062.249360
3,Afghanistan,AFG,2005,1136.123214
4,Afghanistan,AFG,2006,1161.124889
5,Afghanistan,AFG,2007,1284.775213
6,Afghanistan,AFG,2008,1298.143159
7,Afghanistan,AFG,2009,1531.173993
8,Afghanistan,AFG,2010,1614.255001
9,Afghanistan,AFG,2011,1660.739856


In [15]:
gdp_df.to_html('app/templates/gdpData.html')

In [16]:
population_df=pd.read_csv('data/world-population-by-world-regions-post-1820.csv')
population_df.head(20)

,Entity,Code,Year,"Total population (Gapminder, HYDE & UN)"
0,Afghanistan,AFG,1800,3280000
1,Afghanistan,AFG,1801,3280000
2,Afghanistan,AFG,1802,3280000
3,Afghanistan,AFG,1803,3280000
4,Afghanistan,AFG,1804,3280000
5,Afghanistan,AFG,1805,3280000
6,Afghanistan,AFG,1806,3280000
7,Afghanistan,AFG,1807,3280000
8,Afghanistan,AFG,1808,3280000
9,Afghanistan,AFG,1809,3280000


In [17]:
population_df.to_html('app/templates/populationData.html')